In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats as ss
import datetime as dt

from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df = pd.read_csv("/content/drive/MyDrive/ms_wind_curtailment_prediction/curtailment_target_features.csv", sep=";", index_col=0)

**Impute missing values**

In [10]:
# impute missing values for forecast solar with actual solar
for index, row in df.iterrows():
    if pd.isna(row['forecast_solar_MW']):
        df.at[index, 'forecast_solar_MW'] = row['actual_solar_MW']

# impute other missing values by interpolation
columns_to_interpolate = ["wind_speed_m/s",  "wind_direction_degrees", "humidity_percent", "radiation_global_J/m2", "air_temperature_K", "wind_gust_max_m/s", "wind_direction_gust_max_degrees", "forecast_solar_MW", "total_grid_load_MWh", "residual_load_MWh", "pumped_storage_MWh"]
df[columns_to_interpolate] = df[columns_to_interpolate].interpolate(method='linear', limit_direction='both')
df.drop("actual_solar_MW", axis = 1, inplace = True)

## feature lagging

Lagging a time series involves shifting its values forward in time by one or more steps. This means that the observations in the lagged series will appear to have happened later in time compared to the original series. By doing this, we create "lagged" copies of the series, which can be useful for investigating serial dependence or cycles in the time series data. This allows us to incorporate past values of the series into our prediction model, which may help capture patterns or dependencies in the data and improve forecast accuracy.

https://www.kaggle.com/code/ryanholbrook/time-series-as-features

In [13]:
df_lagged = pd.DataFrame(index=df.index)

for feature in df.columns: # dependent variable included!
    df_lagged[feature] = df[feature]
    df_lagged[feature + '_lag1'] = df[feature].shift(1)
    df_lagged[feature + '_lag2'] = df[feature].shift(2)
    df_lagged[feature + '_lag3'] = df[feature].shift(3)

df_lagged.dropna(inplace = True) # maybe better ways
df_lagged.drop(['redispatch_lag1', 'redispatch_lag2', 'redispatch_lag3'], axis=1, inplace = True)
df_lagged.drop(['level_lag1', 'level_lag2', 'level_lag3'], axis=1, inplace = True)

In [14]:
df_lagged.to_csv("/content/drive/MyDrive/ms_wind_curtailment_prediction/lagged_curtailment_target_features.csv", sep=";")

**Lag plots**

A lag plot visually displays a time series by plotting its values against its corresponding lagged values. By examining a lag plot, one can often identify any patterns or dependencies present in the time series data. Autocorrelation, a common measure of serial dependence, quantifies the correlation between a time series and its lagged versions.

When selecting lags to use as features, it's generally unnecessary to include every lag that exhibits a high autocorrelation. Instead, partial autocorrelation provides a more informative measure. It indicates the correlation of a lag while accounting for the influence of all previous lags, revealing the additional correlation contributed by that specific lag. Essentially, partial autocorrelation helps identify the unique influence of each lag on the time series.

In [ ]:
# Plot partial autocorrelation function for each feature
for feature in df.columns:
    plt.figure(figsize=(12, 6))
    plot_pacf(df[feature], lags=10)
    plt.xlabel('Lag')
    plt.ylabel('Partial Autocorrelation')
    plt.title(f'Partial Autocorrelation Plot for {feature}')
    plt.show()

## ANOVA feature selection

ANOVA feature selection is a statistical method used to identify the most important features in a dataset by assessing the relationship between a categorical variable (or group) and a numerical variable.

The basic idea behind ANOVA feature selection is to determine whether there is a statistically significant difference in the mean of the numerical variable across different categories or levels of the categorical variable. This is done by calculating the F-statistic, which measures the ratio of the variance between groups to the variance within groups.

The null hypothesis in ANOVA states that there is no significant difference in the means of the numerical variable across the different categories of the categorical variable. A low p-value obtained from the ANOVA test indicates that there is strong evidence against the null hypothesis, suggesting that at least one group mean is significantly different from the others.



In [ ]:
def anova_pvalues(df, cat_col, num_cols):
    """
    This function calculates the ANOVA p-values (probability of no correlation)
    between a categorical column and multiple numerical columns of a DataFrame.
    """
    p_values = {}
    for num_col in num_cols:
        CategoryGroupLists = df.groupby(cat_col)[num_col].apply(list)
        AnovaResults = ss.f_oneway(*CategoryGroupLists)
        p_value = round(AnovaResults[1], 3)
        p_values[num_col] = p_value

    return p_values

In [ ]:
def anova_pvalue(df,cat_col,num_col):
    """
    This function spits out the anova p-value (probability of no correlation)
    between a categorical column and a numerical column of a dataframe
    """
    CategoryGroupLists = df.groupby(cat_col)[num_col].apply(list)
    AnovaResults = ss.f_oneway(*CategoryGroupLists)
    p_value = round(AnovaResults[1],3)
    return p_value

In [ ]:
# Numeric columns to test
numeric_columns = ['wind_speed_m/s', 'wind_speed_m/s_lag1', 'wind_speed_m/s_lag2',
                  'wind_speed_m/s_lag3', 'wind_direction_degrees',
                  'wind_direction_degrees_lag1', 'wind_direction_degrees_lag2',
                  'wind_direction_degrees_lag3', 'radiation_global_J/m2',
                  'radiation_global_J/m2_lag1', 'radiation_global_J/m2_lag2',
                  'radiation_global_J/m2_lag3', 'air_temperature_K',
                  'air_temperature_K_lag1', 'air_temperature_K_lag2',
                  'air_temperature_K_lag3', 'humidity_percent', 'humidity_percent_lag1',
                  'humidity_percent_lag2', 'humidity_percent_lag3', 'wind_gust_max_m/s',
                  'wind_gust_max_m/s_lag1', 'wind_gust_max_m/s_lag2',
                  'wind_gust_max_m/s_lag3', 'wind_direction_gust_max_degrees',
                  'wind_direction_gust_max_degrees_lag1',
                  'wind_direction_gust_max_degrees_lag2',
                  'wind_direction_gust_max_degrees_lag3', 'forecast_solar_MW',
                  'forecast_solar_MW_lag1', 'forecast_solar_MW_lag2',
                  'forecast_solar_MW_lag3', 'actual_solar_MW', 'actual_solar_MW_lag1',
                  'actual_solar_MW_lag2', 'actual_solar_MW_lag3', 'total_grid_load_MWh',
                  'total_grid_load_MWh_lag1', 'total_grid_load_MWh_lag2',
                  'total_grid_load_MWh_lag3', 'residual_load_MWh',
                  'residual_load_MWh_lag1', 'residual_load_MWh_lag2',
                  'residual_load_MWh_lag3', 'pumped_storage_MWh',
                  'pumped_storage_MWh_lag1', 'pumped_storage_MWh_lag2',
                  'pumped_storage_MWh_lag3']

# Calculate ANOVA p-values
p_values = anova_pvalues(df_lagged, 'redispatch', numeric_columns)

# Print the p-values for each numeric column
for num_col, p_value in p_values.items():
    print(f"p-value {num_col}: {p_value}")

In [ ]:
p_value = anova_pvalue(df_lagged, 'redispatch', 'air_temperature_K')
p_value

0.0

In [ ]:
df_lagged[df_lagged['redispatch'] == 1]

,redispatch,redispatch_lag1,redispatch_lag2,redispatch_lag3,wind_speed_m/s,wind_speed_m/s_lag1,wind_speed_m/s_lag2,wind_speed_m/s_lag3,wind_direction_degrees,wind_direction_degrees_lag1,...,total_grid_load_MWh_lag2,total_grid_load_MWh_lag3,residual_load_MWh,residual_load_MWh_lag1,residual_load_MWh_lag2,residual_load_MWh_lag3,pumped_storage_MWh,pumped_storage_MWh_lag1,pumped_storage_MWh_lag2,pumped_storage_MWh_lag3
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-03-20 11:45:00,1.0,0.0,0.0,0.0,4.05,4.20,4.50,4.40,335.0,340.0,...,1101.15,1091.12,721.43,727.32,733.90,734.53,8.90,9.27,7.05,5.55
2020-03-20 12:00:00,1.0,1.0,0.0,0.0,4.10,4.05,4.20,4.50,340.0,335.0,...,1102.77,1101.15,718.03,721.43,727.32,733.90,1.00,8.90,9.27,7.05
2020-03-20 12:15:00,1.0,1.0,1.0,0.0,3.85,4.10,4.05,4.20,330.0,340.0,...,1100.97,1102.77,711.70,718.03,721.43,727.32,1.15,1.00,8.90,9.27
2020-03-20 12:30:00,1.0,1.0,1.0,1.0,3.60,3.85,4.10,4.05,350.0,330.0,...,1101.83,1100.97,699.10,711.70,718.03,721.43,0.43,1.15,1.00,8.90
2020-03-20 12:45:00,1.0,1.0,1.0,1.0,4.55,3.60,3.85,4.10,350.0,350.0,...,1095.87,1101.83,686.55,699.10,711.70,718.03,0.22,0.43,1.15,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-14 15:45:00,1.0,1.0,1.0,1.0,0.50,0.50,0.45,0.40,105.0,360.0,...,1089.73,1095.73,1014.52,1012.85,1009.70,1011.02,0.17,0.18,0.23,0.77
2023-12-14 16:00:00,1.0,1.0,1.0,1.0,0.50,0.50,0.50,0.45,140.0,105.0,...,1090.17,1089.73,1019.70,1014.52,1012.85,1009.70,0.17,0.17,0.18,0.23
2023-12-14 16:15:00,1.0,1.0,1.0,1.0,0.95,0.50,0.50,0.50,125.0,140.0,...,1087.97,1090.17,1027.85,1019.70,1014.52,1012.85,0.13,0.17,0.17,0.18


A p-value of 0.0 typically indicates strong evidence against the null hypothesis, suggesting that there is a significant relationship between the categorical variable ('redispatch') and the numeric features (along with their lagged versions).

In this case, since all the p-values are 0.0, it suggests that there is a significant association between the categorical variable ('redispatch') and each of the numeric features and their lagged versions. This could imply that these features are potentially important predictors for the 'redispatch' variable.